In [1]:
# !wget https://huggingface.co/datasets/CopyleftCultivars/SemiSynthetic_Composting_Knowledge_For_Agriculture/resolve/main/SemiSynthetic_Composting_Knowledge_For_Agriculture.json?download=true

--2024-02-18 06:24:04--  https://huggingface.co/datasets/CopyleftCultivars/SemiSynthetic_Composting_Knowledge_For_Agriculture/resolve/main/SemiSynthetic_Composting_Knowledge_For_Agriculture.json?download=true
Resolving huggingface.co (huggingface.co)... 2600:9000:234c:f600:17:b174:6d00:93a1, 2600:9000:234c:9800:17:b174:6d00:93a1, 2600:9000:234c:a200:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:234c:f600:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31505 (31K) [text/plain]
Saving to: ‘SemiSynthetic_Composting_Knowledge_For_Agriculture.json?download=true’

SemiSynthetic_Compo 100%[===================>]  30.77K  --.-KB/s    in 0.001s  

2024-02-18 06:24:04 (41.2 MB/s) - ‘SemiSynthetic_Composting_Knowledge_For_Agriculture.json?download=true’ saved [31505/31505]



In [1]:
import json
# import intel_extension_for_pytorch as ipex
import requests
from tokenizers import normalizers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import transformers
import torch
import os
from tqdm.auto import tqdm
import speech_recognition as sr
import pyttsx3

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

def serialize_recipes(data):
    serial = []
    for recipe in data:
        recipe_string = ""
        for key, value in recipe.items():
            recipe_string += ''.join(key) + ":\n" + "".join(value) + "\n"
        serial.append(recipe_string)
    return serial

database = json.load(open("./SemiSynthetic_Composting_Knowledge_For_Agriculture.json download=true", "rb"))
database = serialize_recipes(database)

cuda


In [6]:
VECTOR_DB = "./vector_db.pt"
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
embedder = transformers.pipeline('feature-extraction', model=model, tokenizer=tokenizer)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def embed_data(data):
    embedded = []
    for i in tqdm(data, desc="generating document embeddings: "):
        embedded.append(i)
    return embedded

embedded_database = embed_data(database)

generating document embeddings:   0%|          | 0/51 [00:00<?, ?it/s]

In [36]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        q_features = embedder(query)[0]
        d_features = embedder(doc)[0]
        min_size = min(len(q_features), len(d_features))
        q_features = torch.tensor(q_features[0:-1:len(q_features) - 1//min_size])
        d_features = torch.tensor(d_features[0:-1:len(d_features) - 1//min_size])
        similarity = torch.sum(torch.nn.functional.cosine_similarity(d_features.float(), 
                                                           q_features.float())).item()
        similarities.append(similarity)
    return database[similarities.index(max(similarities))]

def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, pad_token_id=tokenizer.pad_token_id, max_length=300, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]

def generate_RAG(user_input):
    relevant_document = return_response(user_input, embedded_database)
    system_input = f"""You are a farming assitant named spot that makes recommendations for good farming practices. 
                You answer in very short sentences and are very helpful and truthful.
                Formulate your answer using this document: {relevant_document}"""
    return generate_response(system_input, user_input)

In [37]:
def SpeakText(command):
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command) 
    engine.runAndWait()

In [38]:
user_prompt = "Im not sure how to plant my potatoes, can you help me out?"

In [39]:
# generated using rag with a db of farming knowledge on Intel/neural-chat-7b-v3-3
system_input = """You are a farming assitant named spot that makes recommendations for good farming practices. 
                You answer in very short sentences and are very helpful and truthful.
                Formulate your answer using this document: {relevant_document}"""
advice = generate_RAG(user_prompt)
print(advice)
SpeakText(advice)

Sure, here's a step-by-step guide on how to plant potatoes:

1. Choose a potato variety that is suitable for your climate and soil type.
2. Dig a hole that is twice the depth of the potato plant.
3. Fill the hole with potting soil.
4. Place the potato plant in the hole, making sure it is level with the soil.
5. Fill the hole with more potting soil, making sure it is evenly distributed.
6. Water the plant thoroughly.
7. Place a st


In [35]:
# generated using Intel/neural-chat-7b-v3-3
input = f"""You are a farming assitant named spot that makes recommendations for good farming practices. 
                You answer in very short sentences and are very helpful and truthful."""
print(generate_response(input, user_prompt))

Sure, I'd be happy to help you out. Here are some tips for planting potatoes:
1. Choose a good location: Potatoes prefer well-drained soil with a pH between 6.0 and 7.0. Choose a spot that receives at least 6 hours of direct sunlight per day.
2. Dig a hole: Use a shovel or garden fork to dig a hole that is twice the depth of the potato you want to plant. Make sure the hole is slightly larger than the potato.
3. Fill the hole: Fill the hole with a mixture of potting soil and compost. Make sure the soil is moist but not waterlogged.
4. Plant the potato: Place the potato in the hole, making sure it is level with the soil. Firmly press the potato down into the soil.
5. Water the potato: Water the potato thoroughly, making sure it is completely submerged in the soil.
6. Add fertilizer: If you have fertil
